# Residential Energy Use Analysis (RECS 2009)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF

file_path = "/mnt/data/recs2009_public.csv"
df = pd.read_csv(file_path)

## Identify Energy-Related Columns

In [ ]:
df.columns[df.columns.str.contains("KWH|BTU|ENERGY")]

## Average TOTALBTU by Region

In [ ]:
region_labels = {1: "Northeast", 2: "Midwest", 3: "South", 4: "West"}
df_region = df.groupby("REGIONC")["TOTALBTU"].mean().sort_index()
df_region.index = df_region.index.map(region_labels)

plt.figure(figsize=(8, 5))
plt.bar(df_region.index, df_region.values)
plt.ylabel("Average Total Energy Use (BTU)")
plt.title("Average Household Energy Consumption by Region")
plt.grid(axis="y")
plt.tight_layout()
plt.show()

## Region + Housing Type Energy Breakdown

In [ ]:
housing_labels = {
    1: "Mobile Home",
    2: "Single-Family Detached",
    3: "Single-Family Attached",
    4: "Apt (2–4 Units)",
    5: "Apt (5+ Units)"
}
df_grouped = df.groupby(["REGIONC", "TYPEHUQ"])["TOTALBTU"].mean().unstack()
df_grouped.index = df_grouped.index.map(region_labels)
df_grouped.columns = [housing_labels.get(c, str(c)) for c in df_grouped.columns]

df_grouped.plot(kind="bar", figsize=(12, 6))
plt.ylabel("Average Total Energy Use (BTU)")
plt.title("Average Household Energy Use by Region and Housing Type")
plt.grid(axis="y")
plt.tight_layout()
plt.show()

## Heating & Cooling Share by Region

In [ ]:
df_heat_cool = df[["REGIONC", "TOTALBTU", "TOTALBTUSPH", "TOTALBTUCOL"]].dropna()
grouped = df_heat_cool.groupby("REGIONC").mean()
grouped["% Heating"] = (grouped["TOTALBTUSPH"] / grouped["TOTALBTU"]) * 100
grouped["% Cooling"] = (grouped["TOTALBTUCOL"] / grouped["TOTALBTU"]) * 100
grouped.index = grouped.index.map(region_labels)

grouped[["% Heating", "% Cooling"]].plot(kind="bar", figsize=(10, 6))
plt.title("Share of Energy Use for Heating vs Cooling by Region")
plt.ylabel("Percent of Total Household Energy Use")
plt.tight_layout()
plt.show()

## Heating/Cooling Breakdown by Region + Housing Type

In [ ]:
df_full = df[["REGIONC", "TYPEHUQ", "TOTALBTU", "TOTALBTUSPH", "TOTALBTUCOL"]].dropna()
full_grouped = df_full.groupby(["REGIONC", "TYPEHUQ"]).mean()
full_grouped["% Heating"] = (full_grouped["TOTALBTUSPH"] / full_grouped["TOTALBTU"]) * 100
full_grouped["% Cooling"] = (full_grouped["TOTALBTUCOL"] / full_grouped["TOTALBTU"]) * 100

full_grouped = full_grouped.reset_index()
full_grouped["REGIONC"] = full_grouped["REGIONC"].map(region_labels)
full_grouped["TYPEHUQ"] = full_grouped["TYPEHUQ"].map(housing_labels)

heating_by_type = full_grouped.pivot(index="REGIONC", columns="TYPEHUQ", values="% Heating")
cooling_by_type = full_grouped.pivot(index="REGIONC", columns="TYPEHUQ", values="% Cooling")

## Save Visuals

In [ ]:
heating_by_type.plot(kind="bar", figsize=(12, 6))
plt.title("Space Heating Energy Share by Region and Housing Type")
plt.tight_layout()
plt.savefig("/mnt/data/heating_energy_share_by_region.png")
plt.close()

cooling_by_type.plot(kind="bar", figsize=(12, 6))
plt.title("Cooling Energy Share by Region and Housing Type")
plt.tight_layout()
plt.savefig("/mnt/data/cooling_energy_share_by_region.png")
plt.close()

## Export Report to PDF

In [ ]:
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 10, "Residential Energy Use Report (RECS 2009)")
pdf.image("/mnt/data/heating_energy_share_by_region.png", w=180)
pdf.image("/mnt/data/cooling_energy_share_by_region.png", w=180)
pdf.output("/mnt/data/Residential_Energy_Use_Portfolio.pdf")